In [1]:
import discord
import os
from dotenv import load_dotenv

from api_keys import *
from command_handling import *

from DESTINY import basics
from DESTINY import manifestation
from DESTINY import hbfunc

from selenium import webdriver

import bs4
import urllib

import numpy as np
import cv2
import matplotlib.pyplot as plt
import time

import json
import requests
import shutil

import numpy as np

In [2]:
all_data = manifestation.get_all_data(regen=False)

Not regenerating manifest


In [3]:
weapon_dict = manifestation.get_weapon_dict(all_data)

In [4]:
perk_dict = manifestation.get_perk_dict(all_data)

In [5]:
for ep in all_data.keys():
    print(ep)

DestinyPlaceDefinition
DestinyActivityDefinition
DestinyActivityTypeDefinition
DestinyClassDefinition
DestinyGenderDefinition
DestinyInventoryBucketDefinition
DestinyRaceDefinition
DestinyTalentGridDefinition
DestinyUnlockDefinition
DestinyStatGroupDefinition
DestinyFactionDefinition
DestinyVendorGroupDefinition
DestinyRewardSourceDefinition
DestinyItemCategoryDefinition
DestinyDamageTypeDefinition
DestinyActivityModeDefinition
DestinyMedalTierDefinition
DestinyAchievementDefinition
DestinyActivityGraphDefinition
DestinyBondDefinition
DestinyCollectibleDefinition
DestinyDestinationDefinition
DestinyEquipmentSlotDefinition
DestinyEventCardDefinition
DestinyFireteamFinderActivityGraphDefinition
DestinyFireteamFinderActivitySetDefinition
DestinyFireteamFinderLabelDefinition
DestinyFireteamFinderLabelGroupDefinition
DestinyFireteamFinderOptionDefinition
DestinyFireteamFinderOptionGroupDefinition
DestinyStatDefinition
DestinyInventoryItemDefinition
DestinyItemTierTypeDefinition
DestinyLoado

In [6]:
def find_id(search_id):
    for key in all_data.keys():
        if search_id in all_data[key].keys():
            print(key, "\n", json.dumps(all_data[key][search_id], indent=4))

###  Statmax/statmin command:
similar to sametype, but find the one with the highest/lowest stat in a given category

In [7]:
import difflib

Pseudocode for  statmin/statmax:  
ah idk if I wanna bother with types and archetypes ehhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh


In [8]:
def remove_outer_spaces(s:str):
    """
    Remove leading or trailing spaces from string s and  return
    """
    mid_s = ""
    
    # handling leading
    leading=True
    for c in s:
        if leading is True:
            if c == " ":
                continue
            else:
                leading=False
                mid_s+=c
        
        else:
            mid_s += c
            
    final_s_r = ""
    
    # handling trailing
    n = len(mid_s)
    trailing=True
    for i in range(len(mid_s)):
        i_r = n - i - 1
        
        c_r  = mid_s[i_r]
        if trailing is True:
            if c_r == " ":
                continue
            else:
                final_s_r+=c_r
                trailing=False
                
        else:
            final_s_r+=c_r
    
    # reverse
    final_s = ""
    for i in range(len(final_s_r)):
        i_r = len(final_s_r) - i - 1
        c_r = final_s_r[i_r]
        final_s += c_r
    
    
    return final_s


remove_outer_spaces("    ab cde ")

'ab cde'

In [9]:
def statorder(message, args, client, all_data, weapon_dict):
    # need to also specify desired stat, which would come from args...
    weap, stat = " ".join(args).split("&")
    weap = remove_outer_spaces(weap)
    stat = remove_outer_spaces(stat)
    print(weap, stat)
    sametype_string = hbfunc.sametype(message, args, client, all_data, weapon_dict)
    
    for i, line in enumerate(sametype_string.split("\n")):
        # skip the first one
        if i ==0:
            continue
            
        # skip line breaks
        if len(line)==0:
            continue
            
        # if you reach these tiers, end the loop
        if line in ["**RARE**", "**UNCOMMON**",  "**COMMON**"]:
            break
        
        # if it's a title or 
        elif line in ["**EXOTIC**", "**LEGENDARY**"]:
            continue
            
        else:
            # this is an Exotic or Legendary weapon name
            print(line)
            print(hbfunc.weapstat(message, line.split(" "), client, all_data, weapon_dict))
    

statorder(None, ["The", "Palindrome", "&", "zoom"], None, all_data, weapon_dict)

The Palindrome zoom
Ace of Spades
Name: Ace of Spades
Type: Exotic Hand Cannon
Zoom: 14
Rounds Per Minute: 140
Impact: 84
Range: 75
Stability: 40
Magazine: 13
Reload Speed: 42
Handling: 40
Aim Assistance: 70
Recoil Direction: 100
Airborne Effectiveness: 14

Hawkmoon
Name: Hawkmoon
Type: Exotic Hand Cannon
Zoom: 14
Rounds Per Minute: 140
Impact: 84
Range: 52
Stability: 63
Magazine: 8
Reload Speed: 59
Handling: 71
Aim Assistance: 93
Recoil Direction: 96
Airborne Effectiveness: 26

Lumina
Name: Lumina
Type: Exotic Hand Cannon
Zoom: 14
Rounds Per Minute: 140
Impact: 84
Range: 53
Stability: 50
Magazine: 13
Reload Speed: 61
Handling: 71
Aim Assistance: 88
Recoil Direction: 95
Airborne Effectiveness: 27

Thorn
Name: Thorn
Type: Exotic Hand Cannon
Zoom: 14
Rounds Per Minute: 140
Impact: 84
Range: 34
Stability: 55
Magazine: 11
Reload Speed: 40
Handling: 65
Aim Assistance: 85
Recoil Direction: 100
Airborne Effectiveness: 21

Ancient Gospel
Name: Ancient Gospel
Type: Legendary Hand Cannon
Zoom: 1

In [10]:
print(hbfunc.weapstat(None, ["Fatebringer","(Timelost)"], None, all_data, weapon_dict))

Name: Fatebringer (Timelost)
Type: Legendary Hand Cannon
Zoom: 14
Rounds Per Minute: 140
Impact: 84
Range: 46
Stability: 57
Magazine: 10
Reload Speed: 46
Handling: 55
Aim Assistance: 84
Recoil Direction: 98



## Perk combo function
$combo Demolitionist & Adrenaline Junkie --> return weapons that can roll both *in different slots*

In [11]:
from DESTINY.hbfunc import get_closest_gun
#from DESTINY.hbfunc import rolls
import helpers

In [12]:
import inspect  # get function source code
#lines = inspect.getsource(rolls)  
#print(lines)

In [13]:
def rolls_(message, args, client, all_data, weapon_dict, retstr=True):
    """
    get all possible perk rolls for a given weapon
    """

    #weapon = " ".join(args)
    weapon = get_closest_gun(message, args, client, all_data, weapon_dict)[0]

    repeats = 1
    rolls = {}
    
    for idx in reversed(range(len(weapon_dict[weapon]))):
        weaptype = weapon_dict[weapon][idx]['itemTypeAndTierDisplayName']
    
        #print(weaptype)
    
        # for socket (usually barrels, mags, perk1, perk2, etc)
        for socket in weapon_dict[weapon][idx]['sockets']['socketEntries']:
            # if random rolls
            if "randomizedPlugSetHash" in socket.keys():
                randhash = socket['randomizedPlugSetHash']
                sockethash = socket['socketTypeHash']
                socketname = all_data['DestinySocketTypeDefinition'][sockethash]['plugWhitelist'][0]['categoryIdentifier']
            
                # for every perk that goes in that socket for this weapon
                perks = set()
                for perk in all_data['DestinyPlugSetDefinition'][randhash]['reusablePlugItems']:
                    itemhash = perk['plugItemHash']
                    perks.add(all_data['DestinyInventoryItemDefinition'][itemhash]['displayProperties']['name'])
                    
                # save list of perks for each socket name
                if socketname == "frames":
                    socketname = "Main Perk " + str(repeats)
                    repeats += 1
                else:
                    socketname = socketname[0].upper() + socketname[1:]
                    
                rolls[socketname] = list(perks)
        
        if len(rolls) > 0:
            print(idx)
            break

    if retstr is True:
        return(rolls_format(rolls, weapon, weaptype))

    return rolls # can specify retstr = False in parameters if you just want the dict back.

In [14]:
rolls_(None, ["Last", "Rite"], None, all_data, weapon_dict, retstr=False) 

{}

In [15]:
rolls_(None, ["Smite","of","Merain"], None, all_data, weapon_dict, retstr=False) 

0


{'Barrels': ['Chambered Compensator',
  'Smallbore',
  'Arrowhead Brake',
  'Polygonal Rifling',
  'Hammer-Forged Rifling',
  'Corkscrew Rifling',
  'Full Bore',
  'Fluted Barrel',
  'Extended Barrel'],
 'Magazines': ['Accurized Rounds',
  'Tactical Mag',
  'Flared Magwell',
  'Steady Rounds',
  'Alloy Magazine',
  'Appended Mag',
  'Extended Mag'],
 'Main Perk 1': ['Moving Target',
  'Ensemble',
  'Well-Rounded',
  'Focused Fury',
  'Pugilist',
  'Demolitionist',
  'Stats for All'],
 'Main Perk 2': ['Gutshot Straight',
  'Firefly',
  'Adrenaline Junkie',
  'Eye of the Storm',
  'Swashbuckler',
  'Vorpal Weapon',
  'One for All']}

In [16]:
for w in weapon_dict.keys():
    r = rolls_(None, w.split(" "), None, all_data, weapon_dict, retstr=False)
    if len(r)==0:
        print(w)

Classical-42
Countess SA/2
0
0
0
3
Wicked Sister
1
0
Forerunner
1
0
0
1
0
1
0
Telesto
0
Pariah
Psi Termina II
Cadenza-11
Standing Tall
Psi Ferox II
0
0
0
0
0
2
2
Deathbringer
0
0
0
3
1
1
0
0
0
Veles-X
0
The Huckleberry
2
1
2
Belfry Bounty
Maestro-46
0
0
2
0
1
1
0
2
0
0
0
0
0
1
Cuboid ARu
Refrain-23
Sand Wasp-3au
Ros Lysis II
Witherhoard
1
Drang
Rat King
Anarchy
1
0
1
Eyes of Tomorrow
0
0
0
New City
Atalanta-D
Bastion
0
0
1
0
1
The Quickstep
0
0
1
0
1
0
1
0
Quitclaim Shotgun III
0
Eriana's Vow
1
0
3
1
0
0
0
Shock and Awe
Nox Echo III
0
Ghost Primus
0
0
0
Purpose
0
0
0
0
The Mornin' Comes
Scipio-D
3
Headstrong
Helios HC1
0
0
0
0
0
0
0
0
0
0
1
0
Allegro-34
Azimuth DSu
Lamia HC2
Ballyhoo Mk.27
Leviathan's Breath
0
Cloudstrike
Troubadour
Luna Nullis II
Trondheim-LR2
0
Imminent Storm
1
Haunted Earth
0
0
0
1
0
Gentleman Vagabond
1
0
0
0
0
Traitor's Fate
Older Sister III
Home Again
Cydonia-AR1
SUROS Throwback
Jiangshi AR1
Two-Tailed Fox
Komodo-4FR
Foggy Notion
Out of Options
Red Mamba
Somerled

In [17]:
len(weapon_dict["Last Rite"])

1

In [18]:
for socket in weapon_dict["Last Rite"][0]['sockets']['socketEntries']:
    #print("randomizedPlugSetHash" in socket.keys())
    print("reusablePlugSetHash" in socket.keys())
    print(socket.keys())

True
dict_keys(['socketTypeHash', 'singleInitialItemHash', 'reusablePlugItems', 'preventInitializationOnVendorPurchase', 'preventInitializationWhenVersioning', 'hidePerksInItemTooltip', 'plugSources', 'reusablePlugSetHash', 'overridesUiAppearance', 'defaultVisible'])
True
dict_keys(['socketTypeHash', 'singleInitialItemHash', 'reusablePlugItems', 'preventInitializationOnVendorPurchase', 'preventInitializationWhenVersioning', 'hidePerksInItemTooltip', 'plugSources', 'reusablePlugSetHash', 'overridesUiAppearance', 'defaultVisible'])
True
dict_keys(['socketTypeHash', 'singleInitialItemHash', 'reusablePlugItems', 'preventInitializationOnVendorPurchase', 'preventInitializationWhenVersioning', 'hidePerksInItemTooltip', 'plugSources', 'reusablePlugSetHash', 'overridesUiAppearance', 'defaultVisible'])
True
dict_keys(['socketTypeHash', 'singleInitialItemHash', 'reusablePlugItems', 'preventInitializationOnVendorPurchase', 'preventInitializationWhenVersioning', 'hidePerksInItemTooltip', 'plugSourc

In [19]:
# for random rolls:
randomkeys = ['socketTypeHash', 'singleInitialItemHash', 'reusablePlugItems', 'preventInitializationOnVendorPurchase', 'preventInitializationWhenVersioning', 'hidePerksInItemTooltip', 'plugSources', 'randomizedPlugSetHash', 'overridesUiAppearance', 'defaultVisible']

In [20]:
# for set keys:
setkeys = ['socketTypeHash', 'singleInitialItemHash', 'reusablePlugItems', 'preventInitializationOnVendorPurchase', 'preventInitializationWhenVersioning', 'hidePerksInItemTooltip', 'plugSources', 'reusablePlugSetHash', 'overridesUiAppearance', 'defaultVisible']

In [21]:
for r,s in zip(randomkeys, setkeys):
    print(r==s, r,s)

True socketTypeHash socketTypeHash
True singleInitialItemHash singleInitialItemHash
True reusablePlugItems reusablePlugItems
True preventInitializationOnVendorPurchase preventInitializationOnVendorPurchase
True preventInitializationWhenVersioning preventInitializationWhenVersioning
True hidePerksInItemTooltip hidePerksInItemTooltip
True plugSources plugSources
False randomizedPlugSetHash reusablePlugSetHash
True overridesUiAppearance overridesUiAppearance
True defaultVisible defaultVisible
